In [4]:
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from EventDriven.eventScheduler import *
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from EventDriven.backtest import OptionSignalBacktest
%load_ext autoreload
%autoreload 2
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-03-03 00:32:00 trade.helpers.Logging INFO: Logging Root Directory: c:\Users\Zino\python-playground\QuantTools\logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [2]:

def create_datasate(stocks: list, start: str,interval: str, engine: str = 'yf', timewidth = None, timeframe = None, end: str = datetime.today(), return_object = False ):
    dataset = []
    if engine.lower() == 'yf':
        for stock in stocks:
            start = prev_monday(start)
            data2 = yf.download(stock, start = start, end = end, interval=interval, progress = False)

            dataset.append(PTDataset(stock, data2))
    else:
        for stk in stocks:
            stock = Stock(stk)
            data = stock.spot(ts = True, ts_start = '2018-01-01')
            data.rename(columns = {x:x.capitalize() for x in data.columns}, inplace= True)
            data['Timestamp'] = pd.to_datetime(data['Timestamp'], format = '%Y-%m-%d')
            data2 = data.set_index('Timestamp')
            data2 = data2.asfreq('W', method = 'ffill')
            data2 = data2.fillna(0)
            data2['Next_Day_Open'] = data2.Open.shift(-1)
            data2['EMA'] = ta.ma('ema', data2.Close, length = 21).fillna(0)
            dataset.append(PTDataset(stk, data2))
    return dataset if return_object else data2

In [3]:
start, end, interval = '2023-05-29', '2024-05-28','1d'
STOCKS = ['AAPL', 'MSFT','GOOGL', 'AMD', 'AMZN']
dataset = create_datasate(STOCKS, start, interval,end = end , return_object=True)
MAStrat.start_date = pd.to_datetime('1994-03-22')
tt = PTBacktester(dataset, MAStrat, cash =1000, commission = 0.0035)
stats = tt.run()
trades = tt.__trades()
shorts = tt.__trades()[tt.__trades()['Size'] < 0]

YF.download() has changed argument auto_adjust default to True


AttributeError: 'PTBacktester' object has no attribute '_PTBacktester__trades'

In [ ]:
trades_ = trades[:20]
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration,Ticker
0,-8,37,38,120.796297,120.921649,-1.002816,-0.001038,2023-07-24,2023-07-25,None,1 days,GOOGL
0,-9,37,38,110.232833,111.139999,-8.164500,-0.008230,2023-07-24,2023-07-25,None,1 days,AMD
1,7,39,40,130.053799,131.194407,7.984252,0.008770,2023-07-26,2023-07-27,None,1 days,GOOGL
1,-8,39,40,111.518319,111.790001,-2.173458,-0.002436,2023-07-26,2023-07-27,None,1 days,AMD
0,-2,40,41,334.706653,329.164206,11.084894,0.016559,2023-07-27,2023-07-28,None,1 days,MSFT
2,-8,41,42,112.983167,114.160004,-9.414691,-0.010416,2023-07-28,2023-07-31,None,3 days,AMD
0,7,42,43,133.666197,133.550003,-0.813357,-0.000869,2023-07-31,2023-08-01,None,1 days,AMZN
1,-3,43,44,329.506339,329.124772,1.144699,0.001158,2023-08-01,2023-08-02,None,1 days,MSFT
3,-8,43,44,113.860092,119.489998,-45.039246,-0.049446,2023-08-01,2023-08-02,None,1 days,AMD
2,-3,45,46,320.472218,327.398401,-20.778549,-0.021612,2023-08-03,2023-08-04,None,1 days,MSFT


In [5]:
import json
with open('../input/profitable_weights.json', 'r') as f:
    weights = json.load(f)
trades_ = pd.read_csv('../input/profitable_trades.csv').iloc[:, 1:]

w_map = {x: w  * 0.75 for x, w in weights.items()}
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,34.0,504.0,524.0,279.795877,253.010232,-910.711955,-0.095733,2023-07-05,2023-08-02,28 days,TSLA
1,28.0,504.0,526.0,192.240502,185.493749,-188.909083,-0.035095,2023-07-05,2023-08-04,30 days,AAPL
2,17.0,504.0,529.0,336.262811,322.011093,-242.279208,-0.042383,2023-07-05,2023-08-09,35 days,MSFT
3,48.0,504.0,536.0,87.043588,82.000000,-242.092217,-0.057943,2023-07-05,2023-08-18,44 days,AVGO
4,11.0,504.0,583.0,130.695846,122.257034,-92.826927,-0.064568,2023-07-05,2023-10-25,112 days,AMZN
5,178.0,504.0,753.0,42.282471,123.470001,14451.380412,1.920123,2023-07-05,2024-07-01,362 days,NVDA
6,1.0,522.0,522.0,170.283917,168.355807,-1.928110,-0.011323,2023-07-31,2023-07-31,0 days,JNJ
7,5.0,522.0,556.0,332.810769,318.700396,-70.551863,-0.042398,2023-07-31,2023-09-18,49 days,HD
8,5.0,530.0,535.0,239.284572,224.549052,-73.677602,-0.061582,2023-08-10,2023-08-17,7 days,BA
9,5.0,530.0,535.0,53.784257,52.084613,-8.498219,-0.031601,2023-08-10,2023-08-17,7 days,WMT


In [6]:
trades_ = trades_[:5]
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,34.0,504.0,524.0,279.795877,253.010232,-910.711955,-0.095733,2023-07-05,2023-08-02,28 days,TSLA
1,28.0,504.0,526.0,192.240502,185.493749,-188.909083,-0.035095,2023-07-05,2023-08-04,30 days,AAPL
2,17.0,504.0,529.0,336.262811,322.011093,-242.279208,-0.042383,2023-07-05,2023-08-09,35 days,MSFT
3,48.0,504.0,536.0,87.043588,82.000000,-242.092217,-0.057943,2023-07-05,2023-08-18,44 days,AVGO
4,11.0,504.0,583.0,130.695846,122.257034,-92.826927,-0.064568,2023-07-05,2023-10-25,112 days,AMZN


In [53]:
#Backtest class 
evb_backtest = OptionSignalBacktest(trades_)

In [39]:
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .900,
   'dte': 365,
   'moneyness_width': 0.02},
   {'direction': 'short',
   'rel_strike': .80,
   'dte': 365,
   'moneyness_width': 0.02}],

 'name': 'vertical_spread'}
evb_backtest.portfolio.max_price = int(10)

Symbol NVDA not being processed but present in weight_map
Symbol QCOM not being processed but present in weight_map
Symbol MU not being processed but present in weight_map
Symbol GOOG not being processed but present in weight_map
Symbol AMD not being processed but present in weight_map
Symbol BAC not being processed but present in weight_map
Symbol BA not being processed but present in weight_map
Symbol SBUX not being processed but present in weight_map
Symbol DIS not being processed but present in weight_map
Symbol HD not being processed but present in weight_map
Symbol WMT not being processed but present in weight_map
Symbol PFE not being processed but present in weight_map
Symbol JNJ not being processed but present in weight_map
Symbol INTC not being processed but present in weight_map


In [50]:
evb_backtest.bars.signal_df.loc[35, 'MSFT'] = 0

In [ ]:
signals = evb_backtest.bars.signal_df
evb_backtest.bars.signal_df['TSLA', 'AVGO', 'AAPL', 'AMZN'] = 0
evb_backtest.bars.signal_df.loc[0, 'MSFT'] = 0
evb_backtest.bars.signal_df.loc[35, 'MSFT'] = 0
evb_backtest.bars.signal_df.loc[61, 'MSFT'] = 1
evb_backtest.bars.signal_df.loc[63, 'MSFT'] = -1
evb_backtest.bars.signal_df

In [52]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

        Date  TSLA  AAPL  MSFT  AVGO  AMZN
0 2023-07-05     0     0     0     0     0
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Advancing to next date: 2023-07-05 00:00:00
        Date  TSLA  AAPL  MSFT  AVGO  AMZN
1 2023-07-06     0     0     0     0     0
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Advancing to next date: 2023-07-06 00:00:00
        Date  TSLA  AAPL  MSFT  AVGO  AMZN
2 2023-07-07     0     0     0     0     0
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Advancing to next date: 2023-07-07 00:00:00
        Date  TSLA  AAPL  MSFT  AVGO  AMZN
3 2023-07-08     0     0     0     0     0
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Advancing to next date: 2023-07-10 00:00:00
        Date  TSLA  AAPL  MSFT  AVGO  AMZN
4 2023-07-09     0     0     0     0     0
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Advancing to next date: 2023-07-11 00:00:00
        Da

KeyboardInterrupt: 

In [29]:
from pandas.tseries.holiday import USFederalHolidayCalendar
HOLIDAY_SET = set(USFederalHolidayCalendar().holidays(start='2023-09-02', end='2023-09-6').strftime('%Y-%m-%d'))
HOLIDAY_SET

{'2023-09-04'}

In [37]:
order_result = evb_backtest.risk_manager.OrderPicker.get_order('MSFT', '2023-09-04', 'C', 5.0,  {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .900,
   'dte': 365,
   'moneyness_width': 0.02},
   {'direction': 'short',
   'rel_strike': .80,
   'dte': 365,
   'moneyness_width': 0.02}],

 'name': 'vertical_spread'})

print(order_result)

{'result': 'IS_HOLIDAY', 'data': None}


In [30]:
stats.print_stats(30)
print(stream.getvalue())
with open('bactest_data.txt', 'w') as f:
  stream.seek(0)
  f.write(stream.read())
  f.flush()

         3601675 function calls (3575878 primitive calls) in 15.779 seconds

   Ordered by: cumulative time
   List reduced from 1489 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000   15.779    7.889 c:\Users\Zino\python-playground\QuantTools\.venv\lib\site-packages\IPython\core\interactiveshell.py:3517(run_code)
        2    0.000    0.000   15.779    7.889 {built-in method builtins.exec}
        1    0.000    0.000   15.779   15.779 C:\Users\Zino\AppData\Local\Temp\ipykernel_3004\2161208997.py:1(<module>)
        1    0.007    0.007   15.779   15.779 C:\Users\Zino\python-playground\QuantTools\EventDriven\backtest.py:37(run)
       10    0.000    0.000   14.838    1.484 C:\Users\Zino\python-playground\QuantTools\EventDriven\portfolio.py:346(update_signal)
       10    0.000    0.000   14.838    1.484 C:\Users\Zino\python-playground\QuantTools\EventDriven\portfolio.py:263(generate_order)
        5    0.0

In [31]:
evb_backtest.portfolio.plot_portfolio()

AttributeError: 'OptionSignalPortfolio' object has no attribute '_trades'

In [ ]:
trades_df = evb_backtest.portfolio.get_trades().head(5)
trades_df

,Ticker,PnL,ReturnPct,EntryPrice,EntryCommission,EntrySlippage,EntryMarketValue,TotalEntryCost,AuxilaryEntryCost,ExitPrice,ExitCommission,ExitSlippage,ExitMarketValue,TotalExitCost,AuxilaryExitCost,Quantity,EntryTime,ExitTime,Duration,Positions
0,TSLA,-10806.520059,-21.677784,498.506687,6.417,16.736810,11459.236810,11488.807620,-29.570810,390.441487,6.417,16.571196,8986.571196,9003.142393,-16.571196,23,2023-07-05,2023-08-02,28,&L:TSLA20240621C330&S:TSLA20240621C346.67
1,AAPL,3656.985131,7.479087,488.961456,3.627,-10.628066,6352.871934,6349.497867,3.374066,525.531308,3.627,-2.465999,6835.534001,6833.068001,2.465999,13,2023-07-05,2023-08-04,30,&L:AAPL20240621C230&S:AAPL20240621C280
2,MSFT,-10309.332567,-21.247498,485.202187,3.906,-1.075379,6788.924621,6795.661241,-6.736621,382.108862,3.906,-1.569939,5353.430061,5351.860122,1.569939,14,2023-07-05,2023-08-09,35,&L:MSFT20240621C355&S:MSFT20240621C365
3,AMZN,-5391.261254,-10.873383,495.821865,0.837,1.628594,1486.628594,1489.931189,-3.302594,441.909252,0.837,-0.935243,1326.564757,1325.629513,0.935243,3,2023-07-05,2023-10-25,112,&L:AMZN20240621C145&S:AMZN20240621C160


In [ ]:
evb_backtest.portfolio.get_weighted_holdings()s

In [ ]:

pd.set_option('display.max_rows', 10000)
evb_backtest.portfolio.get_all_positions()

long                  short  \
datetime   symbol                                              
2023-07-05 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-06 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-07 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-10 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-11 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-12 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-13 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-14 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-17 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-18 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-19 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-20 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-21 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
 

: 